# Chapter 11: Linear Constrained Optimization

In [161]:
from dataclasses import dataclass
import numpy as np

## Algorithm 11.1

In [195]:
@dataclass
class LinearProgram:
    A : np.ndarray
    b: np.ndarray
    c: np.ndarray

def get_vertex(B, LP):
    A, b, c = LP.A, LP.b, LP.c
    b_inds = np.sort(np.array(B)-1)
    AB = A[:, b_inds]
    xB = np.linalg.solve(AB, b)
    x = np.zeros(len(c))
    x[b_inds] = xB.reshape(-1,)
    return x

## Algorithm 11.2

In [170]:
def edge_transition(LP, B, q):
    A, b, c = LP.A, LP.b, LP.c
    n = A.shape[1]
    b_inds = np.sort(np.array(B)-1)
    n_inds = np.sort(np.setdiff1d(np.arange(0,n,1), np.array(B)-1))
    AB = A[:, b_inds]
    d, xB = np.linalg.solve(AB, A[:, n_inds[q-1]]), np.linalg.solve(AB, b)
 
    p, xq_prime = 0, np.infty

    for i in range(len(d)):
        if d[i] > 0:
            v = xB[i,:]/d[i]
            if v < xq_prime:
                p, xq_prime = i, v
    
    return p, xq_prime

### Example

In [171]:
A= np.array([
    [1, 1, 1, 0],
    [-4, 2, 0, 1]
])
b = np.array([[9], [2]])
c = np.array([[3], [-1], [0], [0]])

problema = LinearProgram(A, b, c)
B = [3,4] #base variables indexes (index start from 1)
q = 2 # entering variable (index start from 1) 
p, xp = edge_transition(problema, B, q)
B[p] = q

print(f"New Base: {B} --- Value of entering variable: {xp[0]}")

New Base: [3, 2] --- Value of entering variable: 1.0


## Algorithm 11.3

In [201]:
def step_lp(B, LP):
    A, b, c = LP.A, LP.b, LP.c
    n = A.shape[1]
    b_inds = np.sort(np.array(B)-1)
    n_inds = np.sort(np.setdiff1d(np.arange(0,n,1), np.array(B)-1))
    
    AB, AV = A[:, b_inds], A[:, n_inds]

    xB = np.linalg.solve(AB, b)
    cB = c[b_inds,:]
    lambda_ = np.linalg.solve(AB.T, cB)
    cV = c[n_inds]
    muV = cV - np.matmul(AV.T, lambda_)

    q, p, xq_prime, delta = -1, 0, np.infty, np.infty
    for i in range(len(muV)):
        if muV[i] < 0:
            pi, xi_prime = edge_transition(LP, B, i+1)
            if np.matmul(muV[i],xi_prime) < delta:
                q, p, xq_prime, delta = i+1, pi, xi_prime, np.matmul(muV[i],xi_prime)

    if q == -1:
        return B, True
    
    if xq_prime == np.infty:
        raise ValueError("unbounded")
    
    j = np.argmax(B == b_inds[p]+1)
    B[j] = n_inds[q-1] +1
    return B, False

### Example

In [200]:
A= np.array([
    [1, 1, 1, 0],
    [-4, 2, 0, 1]
])
b = np.array([[9], [2]])
c = np.array([[3], [-1], [0], [0]])

problema = LinearProgram(A, b, c)
B = [3,4] #base variables indexes (index start from 1)

B, done = step_lp(B, problema)

print(B)
print(done)


[3, 2]
False


## Algorithm 11.4

In [197]:
def minimize_lp(B, LP):
    done = False
    while not done:
        B, done = step_lp(B, LP)
    return B

## Algorithm 11.5

In [198]:
def minimize_lp_init(LP):
    A, b, c = LP.A, LP.b, LP.c
    m, n = A.shape
    z = np.ones((m,1))
    Z = np.diag([1 if i >=0 else -1 for i in b])

    A_prime = np.concatenate((A,Z),axis=1)
    b_prime = b
    c_prime = np.concatenate((np.zeros((n,1)),z),axis=0)

    LP_init = LinearProgram(A_prime, b_prime, c_prime)
    B = np.arange(n+1, m+n+1,1)
    B = minimize_lp(B, LP_init)
    if np.any([True if i > n else False for i in B]):
        print("infeasible")

    return get_vertex(B, LP_init)

### Example

In [199]:
A= np.array([
    [1, 1, 1, 0],
    [-4, 2, 0, 1]
])
b = np.array([[9], [2]])
c = np.array([[3], [-1], [0], [0]])

problema = LinearProgram(A, b, c)
minimize_lp_init(problema)


array([0., 1., 8., 0., 0., 0.])

## Algorithm 11.6

In [ ]:
def dual_certificate(LP, x, mu, epsilon=1e-6):
    A, b, c = LP.A, LP.b, LP.c
    primal_feasible = np.all(np.where(x >= 0, True, False)) and np.all(np.isclose(np.mamtmul(A,x), b, atol=epsilon))
    dual_feasible = np.all(np.where(np.mamtmul(A.T,mu) >= c, True, False))
    return primal_feasible and dual_feasible and np.all(np.isclose(np.dot(c,x), np.dot(b,mu), atol=epsilon))